In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import operator as op
import itertools
import matplotlib.pyplot as plt
import numpy.linalg
import cmath
import multiprocessing
import itertools
import scipy.optimize as optimize
from tabulate import tabulate
from documentCentrality import document_centrality
from networkx import algorithms as ag
%matplotlib inline

In [253]:
def keyword_split(df,key):
    df[key]= df[key].str.split("; ", n = 20, expand = False)  
    return df

In [3]:
def node_key_find(kl,label):
    key=kl[kl['keyword']==label]['id'].iloc[0]
    return key

def node_label_find(kl,key):
    label=kl[kl['id']==key]['keyword'].iloc[0]
    return label    

In [210]:
def nodes_intersection(df,kl,t0,t1,t2):
    prelist=[]
    postlist=[]
    pre_df=df[(df['art_year']>=t0)&(df['art_year']<t1)]
    post_df=df[(df['art_year']>=t1)&(df['art_year']<t2)]
    for index, row in pre_df.iterrows():
        for label in row["keyword"]:
            node_id=node_key_find(kl,label)
            prelist.append(node_id)
    for index, row in post_df.iterrows():
        for label in row["keyword"]:
            node_id=node_key_find(kl,label)
            postlist.append(node_id)
    return set(prelist).intersection(set(postlist))

In [242]:
def build_graph(g,df,kl,nodes,t0,t1):
    g_df=df[(df['art_year']>=t0)&(df['art_year']<t1)]
    #node insert
    for index, row in g_df.iterrows():
        for label in row["keyword"]:
            node_id=node_key_find(kl,label)
            if node_id in nodes:
                if g.has_node(node_id):
                    g.nodes[node_id]['art_id'].add(row["art_id"])
                    g.nodes[node_id]['year'].add(row["art_year"])
                    g.nodes[node_id]['title'].add(row["title"])
                else:
                    art_id={row["art_id"]}
                    year={row["art_year"]}
                    title={row["title"]}
                    g.add_node(node_id,art_id=art_id,year=year,title=title)
    #edge insert
    for index, row in g_df.iterrows():
        edges=list(itertools.combinations(row["keyword"], 2))
        for edge in edges:
            node1=node_key_find(kl,edge[0])
            node2=node_key_find(kl,edge[1])
            if (node1 in nodes) and (node2 in nodes) and (node1!=node2):
                if not g.has_edge(node1,node2):
                    g.add_edge(node1,node2)                
    return g

In [222]:
def node_and_article_feature(df,kl,g,t0,t1):
    f_df=df[(df['art_year']>=t0)&(df['art_year']<t1)]
    article_set=[]
    node_set=[]
    for nd,art in g.node(data='art_id'):
        node_set.append(nd)
        for art_s in art:
            article_set.append(art_s)
    article_set= set(article_set)
    print(len(article_set))
    node_set= set(node_set)
    article_index=list(article_set)
    node_index=list(node_set)
    td=np.zeros((len(g.nodes()), len(article_set)))
    for nd,art in g.node(data='art_id'):
        for art_s in art:
            td[node_index.index(nd)][article_index.index(art_s)]=1
                
    #document centrality feature 
    ca,cn=document_centrality(td,20)
    node_feature=pd.DataFrame({'node_index':node_index,'term_centrality':cn})
    article_feature=pd.DataFrame({'article_index':article_index,'article_centrality_on_node':ca})
    return node_feature,article_feature

In [7]:
def classification_data(df,kl,nodes,g_train,g_test):
    train_labels=[]
    train_rows = list(nx.non_edges(g_train))
    for edge in train_rows:
        if g_test.has_edge(edge[0],edge[1]):
            train_labels.append(1)
        else:
            train_labels.append(0)
    train_data = pd.DataFrame({'row_name':train_rows,'label':train_labels})
    test_data = pd.DataFrame({'row_name':list(set(g_test.edges()))})
    return train_data,test_data

In [8]:
def build_feature_set(df,kl,g,feature_frame,t0,t1):
    #node and article feature
    node_feature,article_feature = node_and_article_feature(df,kl,g,t0,t1)

In [249]:
df = pd.read_csv('dataset/apnea-all,3.csv')
key_list = pd.read_csv('dataset/apnea-distinct_keyword.csv')
article_keywords = pd.read_csv('dataset/apnea-art_id,keyword,key_id,3.csv')
sm_df=df[(df['art_year']<2020)]
sm_df=keyword_split(sm_df)

In [250]:
t0=1991
t1=1992
t2=1993
nodes=nodes_intersection(sm_df,key_list,t0,t1,t2)
g=nx.Graph()
g_train=build_graph(g,sm_df,key_list,nodes,t0,t1)
g=nx.Graph()
g_test=build_graph(g,sm_df,key_list,nodes,t1,t2)

In [254]:
print(len(g_train.edges()),len(g_test.edges()),len(g_train.nodes()),len(g_test.nodes()),len(nodes))


31 20 36 36 36


In [252]:
train_data,test_data=classification_data(sm_df,key_list,nodes,g_train,g_test)
build_feature_set(sm_df,key_list,g_train,train_data,t0,t1)
print(len(train_data),len(train_data[(train_data['label']==1)]))

67
599 11


In [55]:
st=0
end=330
cut=4
tds=train_data
coun2=0
coun=[]
for i,j in tds.iterrows():
    coun1=0
    nb1=set(nx.neighbors(g_train,j[0][0]))
    nb2=set(nx.neighbors(g_train,j[0][1]))
    nb1h2=nb1.copy()
    nb2h2=nb2.copy()
    for n2b1 in nb1h2:
        nb=set(nx.neighbors(g_train,n2b1))
        nb1.update(nb)
    for n2b2 in nb2h2:
        nb=set(nx.neighbors(g_train,n2b2))
        nb2.update(nb)    
    for nnb1 in nb1:
        for nnb2 in nb2:
            if(nnb1==nnb2):
                coun1=coun1+1
    if coun1>0:
        coun2=coun2+1
    coun.append(coun1)  
print(coun2,"/",len(tds))

196350 / 325583


In [ ]:
st=0
end=100000
cut=4
tds=train_data[(train_data['label']==0)]
tds=tds[0:end]
coun2=0
for i,j in tds.iterrows():
    coun1=0
    nb1=set(nx.neighbors(g_train,j[0][0]))
    nb2=set(nx.neighbors(g_train,j[0][1]))
    nb1h2=nb1.copy()
    nb2h2=nb2.copy()
    for n2b1 in nb1h2:
        nb=set(nx.neighbors(g_train,n2b1))
        nb1.update(nb)
    for n2b2 in nb2h2:
        nb=set(nx.neighbors(g_train,n2b2))
        nb2.update(nb)
    for nnb in nb1:
        nnb1=set(nx.neighbors(g_train,nnb))
        for nnnb in nnb1:
            nnnb1=set(nx.neighbors(g_train,nnnb))
            if(j[0][1] in nnnb1):
                coun1=coun1+1
    if coun1>0:
        coun2=coun2+1
print(coun2,"/",len(tds))

In [427]:
st=0
end=330
cut=5
tds=train_data[(train_data['label']==1)]
#tds=tds[0:end]
for jj in range(3,cut):
    count=0
    total=0
    for i,j in tds.iterrows():
        paths = list( nx.all_simple_paths(g_train, source=j[0][0], target=j[0][1],cutoff=jj))
        nb1=set(nx.neighbors(g_train,j[0][0]))
        if len(paths)!=0:
            #print(j[0][0],'+',j[0][1],"----",len(paths))
            count=count+1
            total=total+len(paths)
    print(count,"/",end,"----","cut:",jj,"-----","avg:",total/(jj),"----------")

3.1666666666666665 ---- cut: 3 ----- avg: 8050.666666666667 ----------
3.5757575757575757 ---- cut: 4 ----- avg: 102718.75 ----------


In [266]:
end=4*330
tds=train_data[(train_data['label']==0)]
tds=tds[3*330:end]
for jj in range(3,cut):
    count=0
    total=0
    for i,j in tds.iterrows():
        paths = list( nx.all_simple_paths(g_train, source=j[0][0], target=j[0][1],cutoff=jj))
        if len(paths)!=0:
            #print(j[0][0],'+',j[0][1],"----",len(paths))
            count=count+1
            total=total+len(paths)
    print(count/end,"----","cut:",jj,"-----","avg:",total/(jj),"----------")

1.5909090909090908 ---- cut: 3 ----- avg: 21.666666666666668 ----------


In [438]:
tds=train_data[(train_data['label']==0)]
cm_neigh_len=[]
cm_neigh_list=[]
for i,j in train_data.iterrows():
    coun=0
    cm = list(nx.common_neighbors(g_train, j[0][0],j[0][1]))
    cm_neigh_len.append(len(cm))
    cm_neigh_list.append(cm)

In [443]:
train_data['cm']=cm_neigh_len

In [329]:
st=0
end=10000
cut=4
tds=train_data[(train_data['label']==0)]
#tds=tds[0:end]
coun2=0


In [444]:
def discard_negative(train_data,g_train):
    #pref attach
    pref = nx.preferential_attachment(g_train,list(train_data['row_name']))
    pref = list(pref)
    train_data['pref']=np.array(pref)[:,2]
    cm_neigh_len=[]
    cm_neigh_list=[]
    for i,j in train_data.iterrows():
        coun=0
        cm = list(nx.common_neighbors(g_train, j[0][0],j[0][1]))
        cm_neigh_len.append(len(cm))
        cm_neigh_list.append(cm)
    train_data['cm']=cm_neigh_len

In [22]:
pref = nx.preferential_attachment(g_train,list(train_data['row_name']))
pref = list(pref)
train_data['pref']=np.array(pref)[:,2]

In [23]:
cm_neigh_len=[]
cm_neigh_list=[]
for i,j in train_data.iterrows():
    coun=0
    cm = list(nx.common_neighbors(g_train, j[0][0],j[0][1]))
    cm_neigh_len.append(len(cm))
    cm_neigh_list.append(cm)
train_data['cm']=cm_neigh_len

In [56]:
train_data['cm_hop2']=coun

In [406]:
train_data['soundara']=np.array(soundara)[:,2]

In [92]:
l1= len(train_data[((train_data['pref']>0)&(train_data['cm']>1)&(train_data['cm_hop2']>0))& (train_data['label']==0)])
l2= len(train_data[((train_data['pref']>0)&(train_data['cm']>1)&(train_data['cm_hop2']>0))& (train_data['label']==1)])
print(l1,l2)

6943 296


In [45]:
len(train_data[(train_data['cm_hop2']>0)])

1180

In [74]:
sm_df['art_year'].min()

1980

In [170]:
for i,j in train_data[(train_data['label']==1)][0:20].iterrows():
    print (node_label_find(key_list,j[0][0]),"+",node_label_find(key_list,j[0][1]))
    t1=[]
    t2=[]
    for n1 in g_train[j[0][0]]:
        t1.append(j[0][0])
    for n2 in g_train[j[0][1]]:
        t2.append(g_train[j[0][1]][n2]['title'])
    print(t1)
    print("----------------------------------------------------")

osteotomy + surgery
[10248, 10248]
----------------------------------------------------
osteotomy + distraction
[10248, 10248]
----------------------------------------------------
osteotomy + maxilla
[10248, 10248]
----------------------------------------------------
osteotomy + sleep
[10248, 10248]
----------------------------------------------------
osteotomy + apert syndrome
[10248, 10248]
----------------------------------------------------
osteotomy + obstructive
[10248, 10248]
----------------------------------------------------
malocclusion + overjet
[8202]
----------------------------------------------------
sleep position + children
[14349, 14349, 14349, 14349, 14349]
----------------------------------------------------
sleep position + supine position
[14349, 14349, 14349, 14349, 14349]
----------------------------------------------------
paediatric + morbid obesity
[14348]
----------------------------------------------------
paediatric + anaesthesia
[14348]
-----------------

In [171]:
g_train[10248]

AtlasView({5741: {'art_id': {106857}, 'year': {2001}, 'title': {'Mortised genioplasty in the treatment of obstructive sleep apnea An historical perspective and modification of design'}, 'weight': 1}, 13102: {'art_id': {106857}, 'year': {2001}, 'title': {'Mortised genioplasty in the treatment of obstructive sleep apnea An historical perspective and modification of design'}, 'weight': 1}})

In [257]:
sm_df.groupby('art_year').count()

,art_id,keyword,citation,title,author_name,affiliation_1,affiliation_2,country
art_year,,,,,,,,
1980,3,3,3,3,3,3,3,3
1981,4,4,4,4,4,4,4,4
1982,18,18,18,18,18,18,18,17
1983,11,11,11,11,11,11,10,11
1984,8,8,8,8,8,8,8,8
1985,13,13,13,13,13,13,13,13
1986,26,26,26,26,26,26,25,26
1987,25,25,25,25,25,25,24,24
1988,33,33,33,33,33,33,32,29
